# Track and analyze experimental data

In [ ]:
import lamindb as ln
import pandas as pd
import bionty as bt

ln.nb.header()

This notebook is generated by `testuser2`.

In [ ]:
!lndb login testuser2@lamin.ai --password goeoNJKE61ygbz1vhaCVynGERaRrlviPBVQsjkhz

## Select registered metadata

Let's check which experiments the `testuser1` has registered for project P001:

In [ ]:
ln.db.select.project(external_id="P001").df()

In [ ]:
ln.db.select.experiment(project_id=1).df()

In [ ]:
ln.db.select.readout().df()

## Load in the assay data and perform analysis

In [ ]:
dobject = ln.db.select.dobject(name="schmidt22-crispra-gws-IFNG").one()
dobject

In [ ]:
df_raw = ln.db.load(dobject)

## Perform analysis and ingest results into LaminDB

Let's assume we performed analysis from raw data `df_raw`, now we generated two new data objects:
- df: a new df containing lots of columns of statistics.
- hits: a subsetted dataframe containing hits of the screen.

In [ ]:
df = df_raw.set_index("id")
df.head()

In [ ]:
hits_df = df[df["pos|fdr"] < 0.01].copy()

We'd like to ingest the analysis results in the df to LaminDB with a gene feature model.

In [ ]:
feature_model = bt.Gene(id=bt.lookup.gene_id.symbol)

In [ ]:
ingest = ln.db.Ingest()

In [ ]:
ingest.add(
    df,
    name="schmidt22_crispra_gws_IFNG_analyzed",
).link_features(feature_model, featureset_name="CRISPRa-geneset-1")

In [ ]:
ingest.add(
    hits_df,
    name="schmidt22_crispra_gws_IFNG_hits",
).link_features(feature_model, featureset_name="schmidt22_crispra_gws_IFNG_hits")

In [ ]:
ingest.commit()